# Simulations using Pop2net

## No framework

In [22]:
import pop2net as p2n
import random

class OpinionAgent(p2n.Agent):
    def __init__(self, opinion):
        super().__init__()
        self.opinion = opinion
    
    def add_connection_to_other_cluster(self):
        not_neighbors = [agent for agent in self.env.agents if agent.opinion != self.opinion]
        self.connect(random.choice(not_neighbors))
    
class OpinionCluster(p2n.LocationDesigner):
    def split(self, agent):
        return agent.opinion

class Model:
    def __init__(self):
        self.env = p2n.Environment()
        self.env.add_agents([OpinionAgent(opinion = i%5) for i in range(100)])
        self.env.creator.create_locations(location_designers=[OpinionCluster])
    
    def step(self):
        for agent in self.env.agents:
            agent.add_connection_to_other_cluster()
    

model = Model()
for _ in range(3):
    model.step()
model.env.inspector.plot_agent_network()


## Agentpy

In [23]:
import pop2net as p2n
import agentpy as ap

class OpinionAgent(p2n.Agent, ap.Agent):
    def setup(self, opinion):
        self.opinion = opinion
    
    def add_connection_to_other_cluster(self):
        not_neighbors = self.env.agents.select(self.env.agents.opinion != self.env.agents.opinion) # we can use ap.agentLists now!
        self.connect(self.model.random.choice(not_neighbors))
    
class OpinionCluster(p2n.LocationDesigner):
    def split(self, agent):
        return agent.opinion

class Model(ap.Model):
    def setup(self):
        self.env = p2n.Environment(model=self, framework="agentpy")
        self.env.add_agents([OpinionAgent(model=self, opinion=i%5) for i in range(100)])
        self.env.creator.create_locations(location_designers=[OpinionCluster])
    
    def step(self):
        self.env.agents.add_connection_to_other_cluster() # we can use ap.agentLists now!

model = Model()
model.run(steps=3)
model.env.inspector.plot_agent_network()

Completed: 3 steps
Run time: 0:00:00.127005
Simulation finished


## Mesa

In [24]:
import pop2net as p2n
import mesa

class OpinionAgent(p2n.Agent, mesa.Agent):
    def __init__(self, opinion, model):
        super().__init__(model)
        self.opinion = opinion
    
    def add_connection_to_other_cluster(self):
        not_neighbors = [agent for agent in self.env.agents if agent.opinion != self.opinion]
        self.connect(self.model.random.choice(not_neighbors))
    
class OpinionCluster(p2n.LocationDesigner):
    def split(self, agent):
        return agent.opinion

class Model(mesa.Model):
    def __init__(self, *args, seed = None, rng = None, **kwargs):
        super().__init__(*args, seed=seed, rng=rng, **kwargs)
        self.env = p2n.Environment(model=self, framework="mesa")
        self.env.add_agents([OpinionAgent(model=self, opinion=i%5) for i in range(100)])
        self.env.creator.create_locations(location_designers=[OpinionCluster])
    
    def step(self):
        self.env.agents.shuffle_do("add_connection_to_other_cluster")

model = Model()
for _ in range(3):
    model.step()
model.env.inspector.plot_agent_network()